### 통합 - 숙소 상세 정보, 방 상세정보 크롤링

숙소리스트에서 상품아이디를 가져와 상세정보 페이지에서 숙소상세정보, 방상세정보 크롤링

In [2]:
import requests
import json
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re

In [3]:
# 저장한 데이터 불러오기

acc_list = pd.read_csv(f'../data/여기어때_숙소목록전체.csv')
acc_detail = acc_list
acc_detail.head()

,Unnamed: 0,category,name,score,region,ano,lat,lng,pic_url
0,0,1,강남 더 뮤즈,9.8,강남구 역삼동,1824,37.502530,127.038275,https://image.goodchoice.kr/resize_1000X500x0/...
1,1,1,역삼 호텔 디 아티스트,9.3,강남구 역삼동,5675,37.502415,127.038488,https://image.goodchoice.kr/resize_1000X500x0/...
2,2,1,역삼 리치웰,9.3,강남구 역삼동,3805,37.502973,127.039823,https://image.goodchoice.kr/resize_1000X500x0/...
3,3,1,강남 캠퍼스,9.3,강남구 역삼동,4988,37.497310,127.029190,https://image.goodchoice.kr/resize_1000X500x0/...
4,4,1,역삼 인트로호텔,9.9,강남구 역삼동,72748,37.502754,127.039923,https://image.goodchoice.kr/resize_1000X500x0/...


In [21]:
# 저장한 숙소 목록의 상세정보 데이터 가져오기
address_list = []
pic_list = []
detail_list = []
room_detail = []

for idx in tqdm(range(len(acc_list))):
    cur = acc_list.iloc[idx]
    ano = cur['ano']
    category = cur['category']

    time.sleep(0.1)
    url = f"https://www.goodchoice.kr/product/detail?ano={ano}&sel_date=2023-09-30&sel_date2=2023-10-01"

    resp = requests.get(url)
    soup = BeautifulSoup(resp.content , "html.parser")
    html = soup.select_one("#content")

    images = html.select(".swiper-container li")
    image_list = []

    # 숙소 이미지들 크롤링
    for image in images:
        try:
            image_list.append('https:'+image.select_one('img')['data-src'])
        except:
            continue

    pic_list.append(image_list)

    # 주소
    address =  html.select_one("#content > div.top > div.right > div.info > p.address").text.strip()
    address_list.append(address)
    
    # 숙소 상세정보
    detail  = soup.select_one('.detail_info')
    detail_list.append(str(detail).replace("button", "h3"))

    # 방 상세정보
    rooms = html.select_one('.room_info')

    for room in rooms.select('.room'):
            roomCnt = re.sub(r'[^0-9]', '', room.select_one('.price').select_one('p').text.strip())
            if (roomCnt == "") | (roomCnt == "0"):
                roomCnt = 1

            name = room.select_one('.title').text.strip()


            # 모텔의 경우 인원 정보가 없어서 임의로 2명으로 설정, 방개수 1개 설정
            if category == 1:
                checkIn = room.select('.price').pop().select_one('ul li').text.strip()
                checkOut = room.select('.price').pop().select('ul li')[1].text.strip()
                
                defaultPeople = '2'
                maxPeople = '2'
            elif category == 2:
                defaultPeople = soup.select_one('.dot_txt li').text.strip().split(" ")[0]
                maxPeople = soup.select_one('.dot_txt li').text.strip().split(" ")[3]

                if (len(detail.select("ul")[2].select_one("li").text.strip().split("|")) > 1):
                    checkIn = detail.select("ul")[2].select_one("li").text.strip().split("|")[0].replace("00","")
                    checkOut = detail.select("ul")[2].select_one("li").text.strip().split("|")[1].replace("00","")
                else :
                    checkIn = detail.select("ul")[1].select_one("li").text.strip().split("|")[0].replace("00","")
                    checkOut = detail.select("ul")[1].select_one("li").text.strip().split("|")[1].replace("00","")
            else :
                if re.sub(r'[^0-9]', '', soup.select_one('.dot_txt li').text.strip().split(" ")[0]) != "" :
                    defaultPeople = soup.select_one('.dot_txt li').text.strip().split(" ")[0]
                    maxPeople = soup.select_one('.dot_txt li').text.strip().split(" ")[3]
                else:
                    defaultPeople = soup.select('.dot_txt')[1].select_one("li").text.strip().split(" ")[0]
                    maxPeople = soup.select('.dot_txt')[1].select_one("li").text.strip().split(" ")[3]
                
                if (len(detail.select("ul")[2].select_one("li").text.strip().split("|")) > 1):
                    checkIn = detail.select("ul")[2].select_one("li").text.strip().split("|")[0].replace("00","")
                    checkOut = detail.select("ul")[2].select_one("li").text.strip().split("|")[1].replace("00","")
                else :
                    checkIn = detail.select("ul")[1].select_one("li").text.strip().split("|")[0].replace("00","")
                    checkOut = detail.select("ul")[1].select_one("li").text.strip().split("|")[1].replace("00","")
            


            # room table에서 상품아이디를 primary key로 사용
            room_detail.append ({
                "accommodationId" : ano,
                "name" : name,
                "checkIn" : re.sub(r'[^0-9]', '', checkIn),
                "checkOut" : re.sub(r'[^0-9]', '', checkOut),
                "defaultPeople" : re.sub(r'[^0-9]', '', defaultPeople),
                "maxPeople" : re.sub(r'[^0-9]', '', maxPeople),
                "roomCnt" : roomCnt
            })


100%|██████████| 266/266 [03:24<00:00,  1.30it/s]


In [22]:
# 주소, 사진들, 상세정보 추가

acc_detail['address'] = address_list
acc_detail['pic_urls'] = pic_list
acc_detail['detail'] = detail_list

acc_detail.head()

,category,name,score,region,ano,lat,lng,pic_url,address,pic_urls,detail
0,1,강남 더 뮤즈,9.8,강남구 역삼동,1824,37.502530,127.038275,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 678-9,[https://image.goodchoice.kr/resize_490x348/ad...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
1,1,역삼 호텔 디 아티스트,9.3,강남구 역삼동,5675,37.502415,127.038488,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 678-13,[https://image.goodchoice.kr/resize_490x348/ad...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
2,1,역삼 리치웰,9.3,강남구 역삼동,3805,37.502973,127.039823,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 677-9,[https://image.goodchoice.kr/resize_490x348/ad...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
3,1,강남 캠퍼스,9.3,강남구 역삼동,4988,37.497310,127.029190,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 825-30,[https://image.goodchoice.kr/resize_490x348/ad...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
4,1,역삼 인트로호텔,9.9,강남구 역삼동,72748,37.502754,127.039923,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 677-14,[https://image.goodchoice.kr/resize_490x348/ad...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."


In [23]:
# 상세정보 저장
acc_detail.to_csv(f"../data/여기어때_숙소상세정보전체.csv" ,encoding="utf-8")
acc_detail.head(3)

,category,name,score,region,ano,lat,lng,pic_url,address,pic_urls,detail
0,1,강남 더 뮤즈,9.8,강남구 역삼동,1824,37.502530,127.038275,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 678-9,[https://image.goodchoice.kr/resize_490x348/ad...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
1,1,역삼 호텔 디 아티스트,9.3,강남구 역삼동,5675,37.502415,127.038488,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 678-13,[https://image.goodchoice.kr/resize_490x348/ad...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
2,1,역삼 리치웰,9.3,강남구 역삼동,3805,37.502973,127.039823,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 677-9,[https://image.goodchoice.kr/resize_490x348/ad...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."


In [24]:
# 데이터 프레임으로 변환
room_detail_df = pd.DataFrame(room_detail)

# 방 상세정보 저장
room_detail_df.to_csv(f"../data/여기어때_방상세정보전체.csv" ,encoding="utf-8")
room_detail_df.head(3)

,accommodationId,name,checkIn,checkOut,defaultPeople,maxPeople,roomCnt
0,1824,랜덤객실,23,12,2,2,1
1,1824,로이,23,12,2,2,1
2,1824,샴페인골드,23,12,2,2,1
